In [7]:
import sounddevice as sd
import numpy as np
import librosa
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model("my_model.h5")

# Define the classes and corresponding labels
classes = {"on": 0, "off": 1}

# Define the parameters for feature extraction
sampling_rate = 16000
duration = 1
n_mfcc = 20
n_frames = 100

# Define a function to extract features from an audio signal
def extract_features(signal):
    mfccs = librosa.feature.mfcc(y=signal, sr=sampling_rate, n_mfcc=n_mfcc)
    if mfccs.shape[1] < n_frames:
        mfccs = np.pad(mfccs, ((0, 0), (0, n_frames - mfccs.shape[1])), mode='constant')
    else:
        mfccs = mfccs[:, :n_frames]
    return mfccs.T

# Define a function to record audio from the user's microphone and make a prediction
def predict_from_microphone():
    # Start recording audio from microphone
    print("Recording started...")
    audio = sd.rec(int(duration * sampling_rate), samplerate=sampling_rate, channels=1)
    sd.wait()  # Wait for recording to finish
    print("Recording finished.")

    # Extract features from recorded audio
    features = extract_features(audio[:, 0])

    # Reshape the features to match the input shape of the model
    features = np.reshape(features, (1, -1))

    # Make the prediction
    prediction = model.predict(features)

    # Get the predicted label
    predicted_label = list(classes.keys())[list(classes.values()).index(np.argmax(prediction))]

    # Return the predicted label
    return predicted_label

# Get user input for the label to be predicted
# label = input("Enter the label (on/off): ")

# Make a prediction from the user's microphone and get the predicted label
predicted_label = predict_from_microphone()

print(predicted_label)


Recording started...
Recording finished.
1/1 [==============================] - 0s 78ms/step
off


In [1]:

import sounddevice as sd
import numpy as np
import librosa
from tensorflow.keras.models import load_model
import serial

# Open a serial connection to the Arduino board
ser = serial.Serial('COM3', 9600)  # Replace 'COM3' with the port name of your Arduino board
# Send a test command to the Arduino board
# ser.write(b'test')

# # Wait for the Arduino to respond
# response = ser.readline().decode().rstrip()

# # Print the response from the Arduino
# print("response ",response)


# Load the trained model
model = load_model("my_model.h5")

# Define the classes and corresponding labels
classes = {"on": 0, "off": 1}

# Define the parameters for feature extraction
sampling_rate = 16000
duration = 1
n_mfcc = 20
n_frames = 100

# Define a function to extract features from an audio signal
def extract_features(signal):
    mfccs = librosa.feature.mfcc(y=signal, sr=sampling_rate, n_mfcc=n_mfcc)
    if mfccs.shape[1] < n_frames:
        mfccs = np.pad(mfccs, ((0, 0), (0, n_frames - mfccs.shape[1])), mode='constant')
    else:
        mfccs = mfccs[:, :n_frames]
    return mfccs.T

# Define a function to record audio from the user's microphone and make a prediction
def predict_from_microphone():
    # Start recording audio from microphone
    print("Recording started...")
    audio = sd.rec(int(duration * sampling_rate), samplerate=sampling_rate, channels=1)
    sd.wait()  # Wait for recording to finish
    print("Recording finished.")

    # Extract features from recorded audio
    features = extract_features(audio[:, 0])

    # Reshape the features to match the input shape of the model
    features = np.reshape(features, (1, -1))

    # Make the prediction
    prediction = model.predict(features)

    # Get the predicted label
    predicted_label = list(classes.keys())[list(classes.values()).index(np.argmax(prediction))]

    # Send the predicted label as a command to the Arduino board
    if predicted_label == "on":
        ser.write(b'1')  # Send '1' to turn on the LED
    elif predicted_label == "off":
        ser.write(b'0')  # Send '0' to turn off the LED


    # Return the predicted label
    return predicted_label


predicted_label = predict_from_microphone()
ser.close()
print(predicted_label)

SerialException: could not open port 'COM3': FileNotFoundError(2, 'The system cannot find the file specified.', None, 2)